In [10]:

# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

#Assign paths
canada_hospitalizations_path = Path('Resources/COVID-19_Hospitalizations/Canadian_COVID-19_Hospitalizations_combined.csv')
canada_cpi_path = Path('Resources/Consumer_Price_Index/CPI_Cleaned.csv')
canada_coordinates_path = Path('Resources/Canada_coordinates.csv')

#Read CSV files
canada_hospitalizations_df = pd.read_csv(canada_hospitalizations_path, index_col='Date', infer_datetime_format=True, parse_dates=True)
canada_cpi_df = pd.read_csv(canada_cpi_path, index_col='Date', infer_datetime_format=True, parse_dates=True)
canada_coordinates_df = pd.read_csv(canada_coordinates_path)

#Define function for Canada CPI
def canada_cpi_plot():
    canada_cpi_plot = canada_cpi_df.hvplot(
    x='Date', 
    y='VALUE', 
    groupby=['GEO', 'Products and product groups'],
    title = 'Canada CPI 2019-2021'
    )
    return canada_cpi_plot

#Define function for Canada CPI percent change
def canada_cpi_pct_plot():
    canada_cpi_pct_change = canada_cpi_df['VALUE'].pct_change()
    canada_cpi_pct_change = canada_cpi_df.assign(Percent_Change= canada_cpi_pct_change)
    canada_cpi_pct_plot = canada_cpi_pct_change.hvplot(
    x='Date', 
    y='Percent_Change', 
    groupby=['GEO', 'Products and product groups'],
    title = 'Canada CPI Rate of Change 2019-2021'
    )
    return canada_cpi_pct_plot

#Define function for overlaid hvplot of Canada hospitalizations
def canada_hospitalizations_plot():
    canada_hospitalizations_df.set_index('GEO')
    canada_hospitalizations_plot = canada_hospitalizations_df.hvplot(
        x='Date', 
        groupby=['GEO'],
        title = 'Total Canada COVID-19 Hospitalizations and Per Selected Provinces',
        width=1000,
        height=500
        ).overlay('GEO')
    return canada_hospitalizations_plot

def mapbox_canada_summary(canada_coordinates_df):
    #Summary statistics for Canada and each province
    canada_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Canada'].describe()
    alberta_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Alberta'].describe()
    ontario_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Ontario'].describe()
    saskatchewan_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Saskatchewan'].describe()
    manitoba_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Manitoba'].describe()
    quebec_summary = canada_hospitalizations_df.loc[canada_hospitalizations_df['GEO'] == 'Quebec'].describe()
    #Reassign column name to respective region
    canada_summary = canada_summary.rename(columns = {'Hospitalizations':'Canada'})
    alberta_summary = alberta_summary.rename(columns = {'Hospitalizations':'Alberta'})
    ontario_summary = ontario_summary.rename(columns = {'Hospitalizations':'Ontario'})
    saskatchewan_summary = saskatchewan_summary.rename(columns = {'Hospitalizations':'Saskatchewan'})
    manitoba_summary = manitoba_summary.rename(columns = {'Hospitalizations':'Manitoba'})
    quebec_summary = quebec_summary.rename(columns = {'Hospitalizations':'Quebec'})
    #Concatatenate Summary dataframes
    canada_summary_combined = pd.concat([canada_summary, alberta_summary, ontario_summary, manitoba_summary, saskatchewan_summary, quebec_summary], axis=1, join='inner')
    #Transpose combined dataframe for transpose
    canada_summary_combined = canada_summary_combined.transpose()
    #Assign index name to GEO
    canada_summary_combined.index.names = ['GEO']
    #Set index to GEO
    canada_coordinates_df = canada_coordinates_df.set_index('GEO')
    #Concatenate combined summary dataframe with Coordinates dataframe
    canada_summary_combined_coordinates = pd.concat([canada_summary_combined, canada_coordinates_df], axis=1, join='inner')
    #Reset index for concatenated summary coordinates dataframe
    canada_summary_combined_coordinates.reset_index(inplace=True)
    #Plot summary mapbox plot
    mapbox_canada_summary = px.scatter_mapbox(
        canada_summary_combined_coordinates, 
        lat='lat', 
        lon='lon', 
        color='GEO', 
        size='max', 
        title='Canada & Select Provinces COVID-19 Hospitalizations Summary Statistics',
        hover_data=['GEO', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'],
        zoom=4,
        height=700,
        size_max=100
        )
    mapbox_canada_summary.show()

#Define function for mapbox visualiation of hospitalizations
def mapbox_hospitalizations():
    #Reset index for merge
    canada_hospitalizations_df.reset_index(inplace=True)
    #Merge Canada hospitalizations datadrame with Canada coordinates dataframe
    canada_hospitalizations_coordinates = canada_hospitalizations_df.merge(canada_coordinates_df, how='outer', left_on = 'GEO', right_on = 'GEO')
    #Cast Date column to string for mapbox animation plot
    canada_hospitalizations_coordinates['Date'] = canada_hospitalizations_coordinates['Date'].apply(str)
    #Reset index for mapbox animation plot
    canada_hospitalizations_df.reset_index(inplace=True)
    mapbox_hospitalizations = px.scatter_mapbox(
        canada_hospitalizations_coordinates, 
        lat='lat', 
        lon='lon', 
        color='Hospitalizations', 
        size='Hospitalizations', 
        title='Canada and Select Provinces COVID-19 Hospitalizations - March 9th, 2020 - January 5, 2022',
        color_continuous_scale=px.colors.sequential.Jet,
        hover_data=['GEO'],
        zoom=4,
        height=800,
        size_max=100,
        animation_frame='Date',
        animation_group = 'GEO'
        )
    mapbox_hospitalizations.show()